# Preprocessing TLC data

In [1]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sbs
import geopandas as gpd
import folium

In [2]:
from pyspark.sql import SparkSession

# Create a spark session
spark = (
    SparkSession.builder.appName("ADS Project1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)


24/08/22 23:05:14 WARN Utils: Your hostname, Hanshis-Laptop.local resolves to a loopback address: 127.0.0.1; using 10.12.218.96 instead (on interface en0)
24/08/22 23:05:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/22 23:05:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.conf.set("spark.sql.parquet.compression.codec","gzip")

In [4]:
# Read 2023-2024 TLC data
df_2023 = spark.read.parquet('../data/landing/tlc_data/2023/*.parquet')
df_2024 = spark.read.parquet('../data/landing/tlc_data/2024/*.parquet')

In [5]:
# Read 2023-6 and 2024-5 yellow data
path = "../data/landing/tlc_data"
ydf_2023_6 = spark.read.parquet(path + "/2023/Y-2023-06.parquet")
ydf_2024_5 = spark.read.parquet(path + "/2024/Y-2024-05.parquet")

In [6]:
# Read 2023-6 and 2024-5 green data
path = "../data/landing/tlc_data"
gdf_2023_6 = spark.read.parquet(path + "/2023/G-2023-06.parquet")
gdf_2024_5 = spark.read.parquet(path + "/2024/G-2024-05.parquet")

In [7]:
# Show 2023-6 yellow data 
ydf_2023_6.show(10)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2023-06-01 00:08:48|  2023-06-01 00:29:41|              1|          3.4|         1|                 N|         140|         238|           1|       21.9|  3.5|    0.5|       6.

In [8]:
# Show 2023-6 green data
gdf_2023_6.show(10)

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2023-06-01 00:32:25|  2023-06-01 00:39:06|                 N|         1|          74|          42|              1|         0.84|        7.9|  1.0|    0.

## Exploration 

In [13]:
gdf_2023_6.count()

ConnectionRefusedError: [Errno 61] Connection refused

## Preprocessing

In [19]:
# Filtering yellow taxi data
df_2023_filtered = df_2019_1.filter(df_2019_1["trip_distance"] > 10) 
df_2023_filtered.show(5)

NameError: name 'df_2019_1' is not defined

In [ ]:
df_2019_filtered.where(F.col('passenger_count') > 3).limit(10)

In [2]:
# 合并2019年和2021年的数据
 df_all = df_2019_1.union(df_2021_1)
 df_all.show(10)

IndentationError: unexpected indent (3649347377.py, line 2)

## Add new column ##

In [3]:
df_renamed = df_all.withColumnRenamed("RatecodeID", "Rate_codeID") 
df_renamed.show(5)

NameError: name 'df_all' is not defined

In [4]:
df_dropped = df_all.drop("passenger_count_plus_10")
df_dropped.show(5) 

NameError: name 'df_all' is not defined

In [5]:
df_all.groupBy("VendorID").agg({"passenger_count": "avg", "extra": "max"}).show()

NameError: name 'df_all' is not defined

## Sampling Data

In [6]:
SAMPLE_SIZE = 0.01

In [7]:
df = df_all.sample(SAMPLE_SIZE, seed = 20020223).toPandas() 
df

NameError: name 'df_all' is not defined

In [9]:
df_2019_1.printSchema() 
print(f"数据总量: {df_2019_1.count()}") 
df_2019_1.describe().show()

NameError: name 'df_2019_1' is not defined

## Handling missing data 

In [10]:
df_2019_1 = df_2019_1.dropna()
df_2019_1 = df_2019_1.fillna({'passenger_count': 1, 'trip_distance': 0.0})

NameError: name 'df_2019_1' is not defined

 ## Datatype Conversion

In [11]:
df_2019_1 = df_2019_1.withColumn("passenger_count", col("passenger_count").cast("integer"))
df_2019_1 = df_2019_1.withColumn("tpep_pickup_datetime", col("tpep_pickup_datetime")) 
df_2019_1 = df_2019_1.withColumn("tpep_dropoff_datetime", col("tpep_dropoff_datetime"))  

NameError: name 'df_2019_1' is not defined

## Handling duplicates

In [12]:
df_2019_1 = df_2019_1.dropDuplicates()

NameError: name 'df_2019_1' is not defined

## Anomaly 

In [14]:
trip_distance_min = 0
trip_distance_max = 100
df_2019_1 = df_2019_1.filter((df_2019_1["trip_distance"] > trip_distance_min) & (df_2019_1["trip_distance"] > trip_distance_max))

NameError: name 'df_2019_1' is not defined

## Standardization 

In [15]:
from pyspark.ml.feature import StandardScaler, VectorAssembler
assembler = VectorAssembler(inputCols=["trip_distance", "trip_duration"], outputCol= "features")
df_features = assembler.transform(df_2019_1)
scaler = StandardScaler(inputCol="features", outputCol="scaled_features") 
scaler_model = scaler.fit(df_features)
df_scaled = scaler_model.transform(df_features)

AssertionError: 